In [ ]:
#| default_exp decoders

# Decoders
> Various utilities to decode MARIS dataset from `NetCDF`.

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from pathlib import Path
from netCDF4 import Dataset
import pandas as pd
import numpy as np
from fastcore.basics import patch, store_attr
import fastcore.all as fc

from marisco.configs import (
    NC_DTYPES, 
    NC_VARS, 
    NC_DIM,
    NC_GROUPS,
    lut_path, 
    Enums,
    nc_tpl_path,
    get_time_units
)

## NetCDF to a dictionary of Data Frames

Convert NetCDF file to a dictionary of dataframes (dfs). Convert `time` from seconds since epoch to a date-time format. 

In [ ]:
#| exports
def nc_to_dfs(
    fname: str # Path to NetCDF file
    ) -> dict: # Dictionary with group names as keys and pandas DataFrames as values
    "Convert a NetCDF (with groups) file to a dictionary of dataframes."
    dfs = {}
    
    with Dataset(fname, 'r') as nc:
        # Process each group in the NetCDF file
        for group_name in nc.groups:
            group = nc.groups[group_name]
            
            # Get all variables in the group
            data = {}
            for var_name in group.variables:
                # Skip dimension variables (like 'id')
                if var_name not in group.dimensions:
                    data[var_name] = group.variables[var_name][:]
            
            # Convert to DataFrame
            df = pd.DataFrame(data)
            
            # Convert time from seconds since epoch if present
            if 'time' in df.columns:
                df['time'] = pd.to_datetime(df['time'], unit='s')
                
            dfs[group_name.upper()] = df
    
    return dfs

Example usage:

In [ ]:
#| eval: false
# fname = Path('../../_data/output/190-geotraces-2021.nc')
fname = Path('../../_data/output/tepco.nc')

dfs = nc_to_dfs(fname)

for grp, df in dfs.items():
    print('group:', grp)
    print(f'shape: {df.shape}')
    print(df.head(), '\n')

group: SEAWATER
shape: (21477, 49)
   sample         lon    lat                time  h3  h3_dl  mn54  mn54_dl  \
0       0  141.029999  37.32 2011-03-21 23:15:00 NaN    NaN   NaN      NaN   
1       1  141.029999  37.32 2011-03-22 14:28:00 NaN    NaN   NaN      NaN   
2       2  141.029999  37.32 2011-03-23 13:51:00 NaN    NaN   NaN      NaN   
3       3  141.029999  37.32 2011-03-24 09:30:00 NaN    NaN   NaN      NaN   
4       4  141.029999  37.32 2011-03-25 10:00:00 NaN    NaN   NaN      NaN   

   co58  co58_dl  ...  te132  te132_dl   i132  i132_dl  cs136  cs136_dl  \
0   5.7      7.6  ...    NaN       NaN  160.0     44.0    6.7       4.7   
1   NaN     15.0  ...    NaN       NaN    NaN     88.0    NaN       7.8   
2   NaN      NaN  ...    NaN       NaN  200.0     58.0    NaN       NaN   
3   NaN      NaN  ...    NaN       NaN  120.0     88.0   68.0      49.0   
4   NaN      NaN  ...   13.0       7.4   58.0     22.0    4.4       3.2   

   tbeta  tbeta_dl  talpha  talpha_dl  
0    

## NetCDF Utilities

Return properties of the NetCDF file

In [ ]:
#| exports
def get_netcdf_properties(file_path: str) -> dict:
    """
    Retrieve general properties of a NetCDF file.

    Parameters:
    file_path (str): Path to the NetCDF file.

    Returns:
    dict: A dictionary containing file properties such as size, format, and dimensions.
    """
    properties = {}
    
    file = Path(file_path)
    
    if not file.exists():
        print(f'File not found: {file_path}')
        return properties

    # Get file size
    properties['file_size_bytes'] = file.stat().st_size
    
    # Open the NetCDF file
    with Dataset(file_path, 'r') as nc:
        # Get file format
        properties['file_format'] = nc.file_format

        # Get groups
        properties['groups'] = list(nc.groups.keys())
        
        # Get global attributes
        properties['global_attributes'] = {attr: nc.getncattr(attr) for attr in nc.ncattrs()}
    
    return properties

Example usage:

In [ ]:
#| eval: false
# fname = Path('../files/nc/encoding-test.nc')
# fname = Path('../../_data/output/dump/100-HELCOM-MORS-2018.nc')
#fname = Path('../../_data/output/190-geotraces-2021.nc')
fname = Path('../../_data/output/100-HELCOM-MORS-2024.nc')

properties = get_netcdf_properties(fname)

for key, val in properties.items():
    if isinstance(val, dict):
        print(f"{key}:")
        for sub_key, sub_val in val.items():
            print(f"  {sub_key}: {sub_val}")
    else:
        print(f"{key}: {val}")

file_size_bytes: 864768
file_format: NETCDF4
groups: ['biota', 'seawater', 'sediment']
global_attributes:
  id: TBD
  title: Environmental database - Helsinki Commission Monitoring of Radioactive Substances
  summary: MORS Environment database has been used to collate data resulting from monitoring of environmental radioactivity in the Baltic Sea based on HELCOM Recommendation 26/3.

The database is structured according to HELCOM Guidelines on Monitoring of Radioactive Substances (https://www.helcom.fi/wp-content/uploads/2019/08/Guidelines-for-Monitoring-of-Radioactive-Substances.pdf), which specifies reporting format, database structure, data types and obligatory parameters used for reporting data under Recommendation 26/3.

The database is updated and quality assured annually by HELCOM MORS EG.
  keywords: oceanography, Earth Science > Oceans > Ocean Chemistry> Radionuclides, Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure, Earth Science > Oceans

Return group properties of the NetCDF file

In [ ]:
#| exports
def get_netcdf_group_properties(file_path: str) -> dict:
    """
    Retrieve properties of each group in a NetCDF file, including dimension sizes.

    Parameters:
    file_path (str): Path to the NetCDF file.

    Returns:
    dict: A dictionary containing properties of each group such as variables, dimensions with sizes, and attributes.
    """
    group_properties = {}

    file = Path(file_path)

    if not file.exists():
        print(f'File not found: {file_path}')
        return group_properties

    with Dataset(file_path, 'r') as nc:
        # Iterate over each group in the NetCDF file
        for group_name, group in nc.groups.items():
            # Get dimensions with their sizes
            dimensions = {dim_name: len(dim) for dim_name, dim in group.dimensions.items()}
            
            group_info = {
                'variables': list(group.variables.keys()),
                'dimensions': dimensions,
                'attributes': {attr: group.getncattr(attr) for attr in group.ncattrs()}
            }
            group_properties[group_name] = group_info

    return group_properties


In [ ]:
#| eval: false
# fname = Path('../files/nc/encoding-test.nc')
# fname = Path('../../_data/output/dump/100-HELCOM-MORS-2018.nc')
#fname = Path('../../_data/output/190-geotraces-2021.nc')
fname = Path('../../_data/output/100-HELCOM-MORS-2024.nc')

properties = get_netcdf_group_properties(fname)

for key, val in properties.items():
    if isinstance(val, dict):
        print(f"{key}:")
        for sub_key, sub_val in val.items():
            print(f"  {sub_key}: {sub_val}")
    else:
        print(f"{key}: {val}")

biota:
  variables: ['lon', 'lat', 'smp_depth', 'time', 'nuclide', 'value', 'unit', 'dl', 'bio_group', 'species', 'body_part', 'drywt', 'wetwt']
  dimensions: {'id': 14873}
  attributes: {}
seawater:
  variables: ['lon', 'lat', 'smp_depth', 'tot_depth', 'time', 'nuclide', 'value', 'unit', 'dl', 'filt']
  dimensions: {'id': 20242}
  attributes: {}
sediment:
  variables: ['lon', 'lat', 'tot_depth', 'time', 'area', 'nuclide', 'value', 'unit', 'dl', 'sed_type', 'top', 'bottom']
  dimensions: {'id': 63868}
  attributes: {}


Return variable properties of the NetCDF file

In [ ]:
#| exports
def get_netcdf_variable_properties(file_path: str, as_df: bool = False) -> dict | pd.DataFrame:
    """
    Retrieve properties of variables in each group of a NetCDF file.

    Parameters:
    file_path (str): Path to the NetCDF file
    as_df (bool): If True, returns a pandas DataFrame; if False, returns nested dictionary

    Returns:
    Union[dict, pd.DataFrame]: Properties of variables either as nested dictionary or DataFrame
    """
    var_properties = {}
    
    file = Path(file_path)
    if not file.exists():
        print(f'File not found: {file_path}')
        return var_properties

    with Dataset(file_path, 'r') as nc:
        for group_name, group in nc.groups.items():
            group_vars = {}
            for var_name, var in group.variables.items():
                var_info = {
                    'group': group_name,
                    'variable': var_name,
                    'data_type': var.dtype.str,
                    'dimensions_id': str(var.dimensions),
                    'dimensions_size': str(var.shape),
                }
                # Add variable attributes
                for attr in var.ncattrs():
                    var_info[f'attr_{attr}'] = str(getattr(var, attr))
                    
                group_vars[var_name] = var_info
            var_properties[group_name] = group_vars

    if not as_df:
        return var_properties
    
    # Convert to DataFrame
    rows = []
    for group_name, group_vars in var_properties.items():
        for var_name, var_info in group_vars.items():
            rows.append(var_info)
    
    df = pd.DataFrame(rows)
    
    # Reorder columns to put key information first
    first_cols = ['group', 'variable', 'dimensions_id', 'dimensions_size']
    other_cols = [col for col in df.columns if col not in first_cols]
    df = df[first_cols + other_cols]
    
    return df

In [ ]:
#| eval: false
# fname = Path('../files/nc/encoding-test.nc')
# fname = Path('../../_data/output/dump/100-HELCOM-MORS-2018.nc')
#fname = Path('../../_data/output/190-geotraces-2021.nc')
fname = Path('../../_data/output/100-HELCOM-MORS-2024.nc')

get_netcdf_variable_properties(fname, as_df=True).T

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
group,biota,biota,biota,biota,biota,biota,biota,biota,biota,biota,...,sediment,sediment,sediment,sediment,sediment,sediment,sediment,sediment,sediment,sediment
variable,lon,lat,smp_depth,time,nuclide,value,unit,dl,bio_group,species,...,tot_depth,time,area,nuclide,value,unit,dl,sed_type,top,bottom
dimensions_id,"('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)",...,"('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)"
dimensions_size,"(14873,)","(14873,)","(14873,)","(14873,)","(14873,)","(14873,)","(14873,)","(14873,)","(14873,)","(14873,)",...,"(63868,)","(63868,)","(63868,)","(63868,)","(63868,)","(63868,)","(63868,)","(63868,)","(63868,)","(63868,)"
data_type,<f4,<f4,<f4,<u8,<i8,<f4,<i8,<i8,<i8,<i8,...,<f4,<u8,<i8,<i8,<f4,<i8,<i8,<i8,<f4,<f4
attr_long_name,Measurement longitude,Measurement latitude,Sample depth below seal level,Time of measurement,Nuclide,Activity,Unit,Detection limit,Biota group,Species,...,Total depth below seal level,Time of measurement,Marine area/region id,Nuclide,Activity,Unit,Detection limit,Sediment type,Top depth of sediment layer,Bottom depth of sediment layer
attr_standard_name,longitude,latitude,sample_depth_below_sea_floor,time,nuclide,activity,unit,detection_limit,biota_group_tbd,species,...,total_depth_below_sea_floor,time,area_id,nuclide,activity,unit,detection_limit,sediment_type_tbd,top_depth_of_sediment_layer_tbd,bottom_depth_of_sediment_layer_tbd
attr_units,degrees_east,degrees_north,m,seconds since 1970-01-01 00:00:00.0,NaN,NaN,NaN,NaN,NaN,NaN,...,m,seconds since 1970-01-01 00:00:00.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
attr_axis,NaN,NaN,Z,T,NaN,NaN,NaN,NaN,NaN,NaN,...,Z,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
attr_time_origin,NaN,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Convert NetCDF to OpenRefine CSV


Maris NetCDF files can be converted to OpenRefine CSV files. The OpenRefine CSV files are compatible with the [OpenRefine](https://openrefine.org/) data cleaning tool which are used during the MARIS data cleaning process before loading into the MARIS database.


### WIP - TODO NetCDFOpenRefineDecoder .

1. Include ENUM with NetCDF file. 
2. Can enum (alone) be used to update the netcdf data to openrefine data?
